안산시 데이터 온도 측정지점을 기준으로 3곳으로 나눌 수 있음.

In [41]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium import FeatureGroup
from config import vworld_key
import json

from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd

In [2]:
data1 = pd.read_csv('dataset/1.안산시_기상데이터.csv')

# addr column 값 통일
data1['addr'] = data1['addr'].replace('경기도 안산시단원구 고잔동', '경기도 안산시 단원구 고잔동')

# '날짜' 컬럼을 datetime 형식으로 변환 및 정렬
data1['obsrv_ymd'] = pd.to_datetime(data1['obsrv_ymd'], format='%Y%m%d')
data1 = data1.sort_values(by='obsrv_ymd')

# 이상한 row들 제거
data1 = data1[data1['wnd_drctn'] >= 0.0] # 풍향이 0 이하인 row들 제거, 측정오류로 처리
data1 = data1[(data1['wnd_drctn'] != 0.0) & (data1['wnd_spd'] != 0.0)] # 풍향과 풍속 전부 0인 row들 제거, 측정오류로 처리
data1 = data1[data1['temp'] > -50.0] # 온도가 -50도 이하인 row들 제거, 측정오류로 처리
data1 = data1[data1['hmdt'] >= 0.0] # 습도가 0도 미만인 row들 제거, 측정오류로 처리
data1 = data1[(data1['prcp_tm'] >= 0.0) & (data1['prcp_dy'] >= 0.0)] # 시간 및 누적 강우량 전부 0인 row들 제거, 측정오류로 처리
data1

,obsrv_cd,obsrv_nm,addr,obsrv_ymd,obsrv_tm,lon,lat,heigt,wnd_drctn,wnd_spd,temp,hmdt,atm_prs,atm_prs_si,prcp_tm,prcp_dy
17421,545,안산,경기도 안산시 사동,2019-01-01,17,126.8385,37.2810,5.92,302.4,2.6,-1.4,60.2,-997,-997,0.0,0.0
17430,435,고잔,경기도 안산시 단원구 고잔동,2019-01-01,12,126.8212,37.3243,32.00,316.4,2.7,-2.3,57.8,-997,-997,0.0,0.0
17429,435,고잔,경기도 안산시 단원구 고잔동,2019-01-01,13,126.8212,37.3243,32.00,289.1,2.7,-2.4,63.5,-997,-997,0.0,0.0
17426,435,고잔,경기도 안산시 단원구 고잔동,2019-01-01,14,126.8212,37.3243,32.00,301.1,3.1,-2.2,65.6,-997,-997,0.0,0.0
17425,545,안산,경기도 안산시 사동,2019-01-01,15,126.8385,37.2810,5.92,302.4,1.1,-0.9,62.6,-997,-997,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64208,435,고잔,경기도 안산시 단원구 고잔동,2023-01-31,8,126.8340,37.3326,73.00,116.7,3.4,-1.8,68.0,-997,-997,0.0,0.0
67524,435,고잔,경기도 안산시 단원구 고잔동,2023-01-31,23,126.8340,37.3326,73.00,264.1,3.2,5.7,97.0,-997,-997,0.0,0.0
67525,545,안산,경기도 안산시 사동,2023-01-31,23,126.8385,37.2810,5.92,45.6,0.1,2.6,89.6,-997,-997,0.0,0.0
66428,435,고잔,경기도 안산시 단원구 고잔동,2023-01-31,18,126.8340,37.3326,73.00,213.5,3.4,6.3,73.7,-997,-997,0.0,0.0


In [3]:
data1_1 = data1[(data1['lon'] == 126.8385) & (data1['lat'] == 37.2810)]
data1_2 = data1[(data1['lon'] == 126.8212) & (data1['lat'] == 37.3243)]
data1_3 = data1[(data1['lon'] == 126.8340) & (data1['lat'] == 37.3326)]

In [4]:
# 사용자 정의 함수: 문자열 열을 그대로 반환하는 함수
def keep_string(column):
    return column.iloc[0]  # 첫 번째 값을 반환합니다.

In [5]:
def mean_to(column):
    lst_mean = np.mean(column.tolist())
    if len([i for i in column.tolist() if i >= lst_mean]) > len([i for i in column.tolist() if i <= lst_mean]):
        return sum([i for i in column.tolist() if i >= lst_mean])/len([i for i in column.tolist() if i >= lst_mean])
    elif len([i for i in column.tolist() if i >= lst_mean]) < len([i for i in column.tolist() if i <= lst_mean]):
        return sum([i for i in column.tolist() if i <= lst_mean])/len([i for i in column.tolist() if i <= lst_mean])
    else:
        return lst_mean

In [18]:
result1_1 = data1_1.groupby('obsrv_ymd').agg({'temp': 'mean', 'hmdt': 'mean'}).reset_index()
result1_2 = data1_2.groupby('obsrv_ymd').agg({'temp': 'mean', 'hmdt': 'mean'}).reset_index()
result1_3 = data1_3.groupby('obsrv_ymd').agg({'temp': 'mean', 'hmdt': 'mean'}).reset_index()

In [19]:
def mksubplot(dataframe):
    # 서로 다른 서브플롯을 가진 그림 생성
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('온도', '습도'))

    # 온도 그래프 (서브플롯 1)
    fig.add_trace(go.Scatter(x=dataframe['obsrv_ymd'], y=dataframe['temp'], mode='lines', name='온도 (°C)', line=dict(color='blue')), row=1, col=1)
    fig.update_yaxes(title_text='온도 (°C)', showline=True, linewidth=1, linecolor='red', row=1, col=1, tickvals=list(range(-20, 41, 10)))
    fig.update_xaxes(title_text='날짜', row=1, col=1, showticklabels=True)

    # 습도 그래프 (서브플롯 2)
    fig.add_trace(go.Scatter(x=dataframe['obsrv_ymd'], y=dataframe['hmdt'], mode='lines', name='습도 (%)', line=dict(color='red')), row=2, col=1)
    fig.update_yaxes(title_text='습도 (%)', showline=True, linewidth=1, linecolor='blue', row=1, col=2, tickvals=list(range(0, 101, 10)))
    fig.update_xaxes(title_text='날짜', row=2, col=1, showticklabels=True)

    # 그림 레이아웃 설정
    fig.update_layout(title='온도와 습도 시계열 데이터', showlegend=True, height=500, width=800)

    # Plotly 그래프를 이미지로 저장
    img_bytes = fig.to_image(format="png")

    # 이미지를 base64로 인코딩
    img_base64 = b64encode(img_bytes).decode()

    return img_base64 #base64 형식으로 내보내기

In [20]:
result1_1

,obsrv_ymd,temp,hmdt
0,2019-01-01,-3.600000,64.828571
1,2019-01-02,-4.566667,55.366667
2,2019-01-03,-3.436364,51.718182
3,2019-01-04,-1.133333,65.766667
4,2019-01-05,-4.046667,58.266667
...,...,...,...
1469,2023-01-27,-7.745000,53.405000
1470,2023-01-28,-6.004545,54.231818
1471,2023-01-29,-0.011765,60.111765
1472,2023-01-30,-0.550000,52.300000


In [21]:
result1_2

,obsrv_ymd,temp,hmdt
0,2019-01-01,-4.236364,66.050000
1,2019-01-02,-4.790909,59.895455
2,2019-01-03,-3.683333,52.912500
3,2019-01-04,-2.031579,70.600000
4,2019-01-05,-2.495455,46.731818
...,...,...,...
798,2021-05-09,14.743478,61.578261
799,2021-05-10,11.652174,90.956522
800,2021-05-11,17.570833,56.341667
801,2021-05-12,20.675000,38.137500


### 지점별 온도시계열변화 시각화

In [22]:
# import io
# from PIL import Image
from base64 import b64encode

coordinate_lst = [[37.2810, 126.8385], [37.3243, 126.8212], [37.3326, 126.8340]]
img_base64_lst = [mksubplot(result1_1), mksubplot(result1_2), mksubplot(result1_3)]

# Folium 지도 생성
m = folium.Map(location=[37.287, 126.8385], zoom_start=12)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 이미지를 팝업에 추가
for coordinate,base64 in zip(coordinate_lst, img_base64_lst):
    popup = folium.Popup(folium.Html(f'<img src="data:image/png;base64,{base64}">', script=True), max_width=800)
    marker = folium.Marker(location=coordinate, popup=popup)
    marker.add_to(m)

# 지도를 화면에 표시
m.save('weather_map.html')  # HTML 파일로 저장

## 3.안산시_가로수 현황 시각화

In [28]:
# GeoJSON 파일 불러오기
with open('dataset/3.안산시_가로수현황.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
street_trees_df = pd.json_normalize(geojson_data['features'])
street_trees_df['geometry.coordinates'] = street_trees_df['geometry.coordinates'].apply(lambda x : x[0])
street_trees_df['lon'] = street_trees_df['geometry.coordinates'].apply(lambda x : x[0])
street_trees_df['lat'] = street_trees_df['geometry.coordinates'].apply(lambda x : x[1])
street_trees_df

,type,properties.HJD_CDE,properties.BJD_CDE,geometry.type,geometry.coordinates,lon,lat
0,Feature,4127357000,4127310400,MultiPoint,"[126.77130981479182, 37.29558311978244]",126.771310,37.295583
1,Feature,4127357000,4127310400,MultiPoint,"[126.77122642985587, 37.29560702529468]",126.771226,37.295607
2,Feature,4127357000,4127310400,MultiPoint,"[126.77114090430125, 37.29563016973914]",126.771141,37.295630
3,Feature,4127357000,4127310400,MultiPoint,"[126.77105525924954, 37.2956555124169]",126.771055,37.295656
4,Feature,4127357000,4127310400,MultiPoint,"[126.77096827268889, 37.2956806361881]",126.770968,37.295681
...,...,...,...,...,...,...,...
68838,Feature,4127357000,4127310400,MultiPoint,"[126.73935732916034, 37.30140089565829]",126.739357,37.301401
68839,Feature,4127357000,4127310400,MultiPoint,"[126.73939465578442, 37.301468384206125]",126.739395,37.301468
68840,Feature,4127357000,4127310400,MultiPoint,"[126.73946116266923, 37.30160096460182]",126.739461,37.301601
68841,Feature,4127357000,4127310400,MultiPoint,"[126.73943989009794, 37.30203909055725]",126.739440,37.302039


### folium.LayerControl() 기억해두기

In [32]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
center_lat = np.mean(street_trees_df['lat'])
center_lon = np.mean(street_trees_df['lon'])
m = folium.Map(location=[center_lat, center_lon], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 마커 클러스터 생성
marker_cluster = MarkerCluster().add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in street_trees_df.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3,
        fill=True,
    )
    marker.add_to(marker_cluster)

# 지도 저장
m.save('street_trees_clustering.html')

### 도시공원 현황

In [24]:
park_df = pd.read_csv('dataset/4.안산시_도시공원현황.csv')
park_df

,park_nm,park_gbn,address,area,lon,lat
0,각골공원,근린공원,경기도 안산시 상록구 본오동 45-6,49303.0,126.874151,37.291292
1,반월공원,근린공원,경기도 안산시 상록구 본오동 723,44732.4,126.869552,37.289870
2,상록수공원,문화공원,경기도 안산시 상록구 본오동 879-4,12325.8,126.862742,37.299643
3,본오공원,근린공원,경기도 안산시 상록구 본오동 산39,123747.0,126.868576,37.300955
4,오목골공원,근린공원,경기도 안산시 상록구 본오동 산80-1,47210.0,126.860578,37.288462
...,...,...,...,...,...,...
190,도토리공원,소공원,경기도 안산시 단원구 고잔동 782-5,4731.0,126.832012,37.307241
191,원곡3문화공원,문화공원,경기도 안산시 단원구 원곡동 852,3645.0,126.803681,37.324755
192,원곡3소공원,소공원,경기도 안산시 단원구 원곡동 854,5507.0,126.804429,37.323728
193,초지1소공원,소공원,경기도 안산시 단원구 초지동 595-3,4864.0,126.806747,37.322585


In [42]:
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'}
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 좌표계 epsg : 4326
    return DataFrame

In [45]:
park_df = geo_transform(park_df)
park_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,park_nm,park_gbn,address,area,lon,lat,geometry
0,각골공원,근린공원,경기도 안산시 상록구 본오동 45-6,49303.0,126.874151,37.291292,POINT (126.87415 37.29129)
1,반월공원,근린공원,경기도 안산시 상록구 본오동 723,44732.4,126.869552,37.289870,POINT (126.86955 37.28987)
2,상록수공원,문화공원,경기도 안산시 상록구 본오동 879-4,12325.8,126.862742,37.299643,POINT (126.86274 37.29964)
3,본오공원,근린공원,경기도 안산시 상록구 본오동 산39,123747.0,126.868576,37.300955,POINT (126.86858 37.30096)
4,오목골공원,근린공원,경기도 안산시 상록구 본오동 산80-1,47210.0,126.860578,37.288462,POINT (126.86058 37.28846)
...,...,...,...,...,...,...,...
190,도토리공원,소공원,경기도 안산시 단원구 고잔동 782-5,4731.0,126.832012,37.307241,POINT (126.83201 37.30724)
191,원곡3문화공원,문화공원,경기도 안산시 단원구 원곡동 852,3645.0,126.803681,37.324755,POINT (126.80368 37.32476)
192,원곡3소공원,소공원,경기도 안산시 단원구 원곡동 854,5507.0,126.804429,37.323728,POINT (126.80443 37.32373)
193,초지1소공원,소공원,경기도 안산시 단원구 초지동 595-3,4864.0,126.806747,37.322585,POINT (126.80675 37.32258)


In [54]:
# CCTV 반경 50m 그리기

# map 생성
m = folium.Map(location=[37.291292, 126.874151],  zoom_start=11, tiles='CartoDB positron')

for _, row in park_df.iterrows() :
    radius = row['area'] ** 0.5  # 면적(m^2)을 미터로 환산하여 radius로 설정
    folium.Circle(location=(row['lat'], row['lon']), radius=radius, color='#FF580B',
            fill='#FF580B').add_to(m)
    
m.save('park_area_radius.html')

### 하천현황

In [ ]:
# GeoJSON 파일 불러오기
with open('dataset/5.안산시_하천현황.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
river_df = pd.json_normalize(geojson_data['features'])
river_df

In [ ]:
river_df['geometry'] = river_df['geometry.coordinates'].apply(lambda x : Polygon(x[0][0]))
river_df

In [103]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[37.37105949652944, 126.88130200939035], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in river_df.iterrows():
    folium.GeoJson(row['geometry'].__geo_interface__).add_to(m)
    
m.save('river_area.html')

### 토지피복현황도

In [ ]:
# GeoJSON 파일 불러오기
with open('dataset/6.안산시_토지피복현황도.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
land_cover_df = pd.json_normalize(geojson_data['features'])
land_cover_df['geometry'] = land_cover_df['geometry.coordinates'].apply(lambda x : Polygon(x[0][0]))
land_cover_df['properties.피복-대분류'] = land_cover_df['properties.피복-대분류'].apply(lambda x : x.split('. ')[1])
land_cover_df['properties.피복-중분류'] = land_cover_df['properties.피복-중분류'].apply(lambda x : x.split('. ')[1])
land_cover_df['properties.피복-소분류'] = land_cover_df['properties.피복-소분류'].apply(lambda x : x.split('. ')[1])
land_cover_df

In [141]:
# 열 'properties.피복-대분류' 값을 기준으로 다른 색상을 매핑하는 함수 생성
def get_color(feature):
    category = feature['properties.피복-대분류']
    if category == '불투수':
        return 'darkred'
    elif category == '투수':
        return 'yellow'
    elif category == '녹지':
        return 'darkgreen'
    else:
        return 'darkblue'

# 지도의 중심 좌표 설정
m = folium.Map(location=[37.37105949652944, 126.88130200939035], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in land_cover_df.iterrows():
    color = get_color(row)
    popup_text = f"{row['properties.피복-소분류']}({row['properties.피복-대분류']} - {row['properties.피복-중분류']})"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {'fillColor': color, 'color': 'black', 'weight': 0.5}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=200))  # 팝업 메시지 추가
    
m.save('land_cover.html')

### 현존 식생도

In [ ]:
# GeoJSON 파일 불러오기
with open('dataset/7.안산시_현존식생도.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
vegetation_map_df = pd.json_normalize(geojson_data['features'])
vegetation_map_df['geometry'] = vegetation_map_df['geometry.coordinates'].apply(lambda x : Polygon(x[0][0]))
vegetation_map_df['properties.현존식생유형_소분류'] = vegetation_map_df['properties.현존식생유형_소분류'].apply(lambda x : x.split('. ')[1])
vegetation_map_df

In [142]:
# 열 'properties.피복-대분류' 값을 기준으로 다른 색상을 매핑하는 함수 생성
def get_color(feature):
    category = feature['properties.현존식생유형_구분']
    if category == '비식생':
        return 'darkred'
    else:
        return 'darkgreen'

# 지도의 중심 좌표 설정
m = folium.Map(location=[37.37105949652944, 126.88130200939035], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in vegetation_map_df.iterrows():
    color = get_color(row)
    popup_text = f"{row['properties.현존식생유형_구분']}(properties.현존식생유형_소분류)"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {'fillColor': color, 'color': 'black', 'weight': 0.5}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=200))  # 팝업 메시지 추가
    
m.save('vegetation_map.html')

### 연속지적도

In [159]:
def make_geo(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [161]:
# GeoJSON 파일 불러오기
with open('dataset/8.안산시_연속지적도.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
registration_map_df = pd.json_normalize(geojson_data['features'])
registration_map_df['geometry'] = registration_map_df['geometry.coordinates'].apply(lambda x : make_geo(x))
registration_map_df

,type,properties.jibun,properties.pnu,geometry.type,geometry.coordinates,geometry
0,Feature,537-6대,4127110100105370006,Polygon,"[[[126.86455223208655, 37.316952069431515], [1...",POLYGON ((126.86455223208655 37.31695206943151...
1,Feature,559-13대,4127110100105590013,Polygon,"[[[126.86179206428146, 37.31692604694183], [12...",POLYGON ((126.86179206428146 37.31692604694183...
2,Feature,541-7대,4127110100105410007,Polygon,"[[[126.86344840613782, 37.3169206925515], [126...","POLYGON ((126.86344840613782 37.3169206925515,..."
3,Feature,559-10대,4127110100105590010,Polygon,"[[[126.8613822727095, 37.316528570263664], [12...",POLYGON ((126.8613822727095 37.316528570263664...
4,Feature,540-4대,4127110100105400004,Polygon,"[[[126.86422893347417, 37.31662768579499], [12...",POLYGON ((126.86422893347417 37.31662768579499...
...,...,...,...,...,...,...
65071,Feature,105-1도,4127110600101050001,Polygon,"[[[126.84645334884664, 37.35043177565401], [12...",POLYGON ((126.84645334884664 37.35043177565401...
65072,Feature,91-2 도,4127110600100910002,Polygon,"[[[126.84695039768427, 37.35290009656722], [12...",POLYGON ((126.84695039768427 37.35290009656722...
65073,Feature,64-10 도,4127110600100640010,Polygon,"[[[126.84644963809912, 37.35397018693188], [12...",POLYGON ((126.84644963809912 37.35397018693188...
65074,Feature,90-3 도,4127110600100900003,Polygon,"[[[126.84706122294956, 37.353902509430256], [1...",POLYGON ((126.84706122294956 37.35390250943025...


In [162]:
# 열 'properties.피복-대분류' 값을 기준으로 다른 색상을 매핑하는 함수 생성


# 지도의 중심 좌표 설정
m = folium.Map(location=[37.37105949652944, 126.88130200939035], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in vegetation_map_df.iterrows():
    folium.GeoJson(
        row['geometry'].__geo_interface__,
    ).add_to(m)
    
m.save('registration_map.html')

### 토지이용현황도

In [175]:
# GeoJSON 파일 불러오기
with open('dataset/9.안산시_토지이용현황도.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
land_use_df = pd.json_normalize(geojson_data['features'])
land_use_df['geometry'] = land_use_df['geometry.coordinates'].apply(lambda x : Polygon(x[0][0]))
land_use_df['properties.이용현황-대분류'] = land_use_df['properties.이용현황-대분류'].apply(lambda x : x.split('. ')[1])
land_use_df['properties.이용현황-중분류'] = land_use_df['properties.이용현황-중분류'].apply(lambda x : x.split('. ')[1])
land_use_df['properties.이용현황-소분류'] = land_use_df['properties.이용현황-소분류'].apply(lambda x : x.split('. ')[1])
land_use_df

,type,properties.이용현황-대분류,properties.이용현황-중분류,properties.이용현황-소분류,geometry.type,geometry.coordinates,geometry
0,Feature,산림지역,활엽수림,활엽수림,MultiPolygon,"[[[[126.84374987651165, 37.36017593517266, 0.0...",POLYGON Z ((126.84374987651165 37.360175935172...
1,Feature,산림지역,활엽수림,활엽수림,MultiPolygon,"[[[[126.84992693171938, 37.3623689488233, 0.0]...",POLYGON Z ((126.84992693171938 37.362368948823...
2,Feature,산림지역,활엽수림,활엽수림,MultiPolygon,"[[[[126.85079467412422, 37.35946627673895, 0.0...",POLYGON Z ((126.85079467412422 37.359466276738...
3,Feature,산림지역,활엽수림,활엽수림,MultiPolygon,"[[[[126.84374987651165, 37.36017593517266, 0.0...",POLYGON Z ((126.84374987651165 37.360175935172...
4,Feature,초지,인공초지,묘지,MultiPolygon,"[[[[126.85092416789732, 37.361073411155786, 0....",POLYGON Z ((126.85092416789732 37.361073411155...
...,...,...,...,...,...,...,...
7113,Feature,시가화 건조지역,교통지역,기타 교통·통신시설,MultiPolygon,"[[[[126.80108601043415, 37.342478214261135, 0....",POLYGON Z ((126.80108601043415 37.342478214261...
7114,Feature,시가화 건조지역,교통지역,도로,MultiPolygon,"[[[[126.75033331114372, 37.328385629553566, 0....",POLYGON Z ((126.75033331114372 37.328385629553...
7115,Feature,시가화 건조지역,문화체육휴양지역,문화체육휴양시설,MultiPolygon,"[[[[126.91384264493489, 37.29652770258018, 0.0...",POLYGON Z ((126.91384264493489 37.296527702580...
7116,Feature,농업지역,논,경지정리가 안 된 논,MultiPolygon,"[[[[126.9238324319318, 37.28543735467033, 0.0]...",POLYGON Z ((126.9238324319318 37.2854373546703...


In [173]:
# 열 'properties.피복-대분류' 값을 기준으로 다른 색상을 매핑하는 함수 생성
def get_color(feature):
    category = feature['properties.이용현황-대분류']
    if category == '산림지역':
        return 'darkgreen'
    elif category == '초지':
        return 'green'
    elif category == '나지':
        return 'yellow'
    elif category == '시가화 건조지역':
        return 'darkred'
    elif category == '농업지역':
        return 'green'
    else:
        return 'darkblue'

# 지도의 중심 좌표 설정
m = folium.Map(location=[37.37105949652944, 126.88130200939035], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in land_use_df.iterrows():
    color = get_color(row)
    popup_text = f"{row['properties.이용현황-소분류']}({row['properties.이용현황-대분류']} - {row['properties.이용현황-중분류']})"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {'fillColor': color, 'color': 'black', 'weight': 0.5}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=200))  # 팝업 메시지 추가
    
m.save('land_use.html')

### 건물일반공간정보

In [174]:
# GeoJSON 파일 불러오기
with open('dataset/10.안산시_GIS건물일반공간정보_.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
building_df = pd.json_normalize(geojson_data['features'])
# building_df['geometry'] = building_df['geometry.coordinates'].apply(lambda x : Polygon(x[0][0]))
# building_df['properties.이용현황-대분류'] = building_df['properties.이용현황-대분류'].apply(lambda x : x.split('. ')[1])
# building_df['properties.이용현황-중분류'] = building_df['properties.이용현황-중분류'].apply(lambda x : x.split('. ')[1])
# building_df['properties.이용현황-소분류'] = building_df['properties.이용현황-소분류'].apply(lambda x : x.split('. ')[1])
building_df

,type,properties.pnu,properties.emd_cd,properties.emd_nm,properties.s_land_cd,properties.s_land_nm,properties.jibun,properties.bldg_nm,properties.bldg_dong,properties.mn_sb_cd,...,properties.bc_rat,properties.bldg_st_cd,properties.bldg_st_nm,properties.bdtyp_cd,properties.bdtyp_nm,properties.bldg_hght,properties.grnd_flr,properties.ugrnd_flr,geometry.type,geometry.coordinates
0,Feature,4127310900110890017,4127310900,경기도 안산시 단원구 선부동,1,일반,1089-17,None,None,0,...,51.64,19,기타조적구조,01000,단독주택,10.60,3,1,Polygon,"[[[126.81715695809969, 37.34303486797774], [12..."
1,Feature,4127310900110890002,4127310900,경기도 안산시 단원구 선부동,1,일반,1089-2,None,None,0,...,59.43,19,기타조적구조,01000,단독주택,10.20,3,1,Polygon,"[[[126.81702391200503, 37.34307340452085], [12..."
2,Feature,4127310900110890018,4127310900,경기도 안산시 단원구 선부동,1,일반,1089-18,None,None,0,...,53.92,19,기타조적구조,01000,단독주택,9.90,3,0,Polygon,"[[[126.81701517425486, 37.34303770143117], [12..."
3,Feature,4127310900110890001,4127310900,경기도 안산시 단원구 선부동,1,일반,1089-1,None,None,0,...,59.57,19,기타조적구조,01000,단독주택,10.00,3,0,Polygon,"[[[126.81675983572548, 37.34322111138423], [12..."
4,Feature,4127310200106980006,4127310200,경기도 안산시 단원구 와동,1,일반,698-6,None,None,0,...,59.87,21,철근콘크리트구조,01000,단독주택,7.90,3,0,Polygon,"[[[126.83020963334549, 37.33340944610932], [12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32616,Feature,4127310200106960003,4127310200,경기도 안산시 단원구 와동,1,일반,696-3,None,None,0,...,59.25,21,철근콘크리트구조,02000,공동주택,12.75,4,0,Polygon,"[[[126.8292448694999, 37.33343229200078], [126..."
32617,Feature,4127310100107810000,4127310100,경기도 안산시 단원구 고잔동,1,일반,781,푸르지오1차아파트,109동,0,...,0.00,21,철근콘크리트구조,02000,공동주택,42.96,15,1,Polygon,"[[[126.83504417395476, 37.30547134902931], [12..."
32618,Feature,4127310100107810000,4127310100,경기도 안산시 단원구 고잔동,1,일반,781,푸르지오1차아파트,상가동,0,...,0.00,21,철근콘크리트구조,02000,공동주택,7.80,2,0,Polygon,"[[[126.83260846467113, 37.307237245702], [126...."
32619,Feature,4127110300115530000,4127110300,경기도 안산시 상록구 사동,1,일반,1553,숲속마을아파트,상가동,0,...,0.00,21,철근콘크리트구조,02000,공동주택,10.60,2,1,Polygon,"[[[126.8390796390443, 37.30415659903055], [126..."


### 도로명주소

In [176]:
# GeoJSON 파일 불러오기
with open('dataset/11.안산시_도로명주소(도로).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_df = pd.json_normalize(geojson_data['features'])
# roadname_df['geometry'] = roadname_df['geometry.coordinates'].apply(lambda x : Polygon(x[0][0]))
# roadname_df['properties.이용현황-대분류'] = roadname_df['properties.이용현황-대분류'].apply(lambda x : x.split('. ')[1])
# roadname_df['properties.이용현황-중분류'] = roadname_df['properties.이용현황-중분류'].apply(lambda x : x.split('. ')[1])
# roadname_df['properties.이용현황-소분류'] = roadname_df['properties.이용현황-소분류'].apply(lambda x : x.split('. ')[1])
roadname_df

,type,properties.SIG_CD,properties.RDS_MAN_NO,properties.RN,properties.RN_CD,properties.ENG_RN,properties.NTFC_DE,properties.WDR_RD_CD,properties.ROA_CLS_SE,properties.RDS_DPN_SE,...,properties.ROAD_LT,properties.BSI_INT,properties.ALWNC_RESN,properties.ALWNC_DE,properties.MVM_RES_CD,properties.MVMN_RESN,properties.MVMN_DE,properties.OPERT_DE,geometry.type,geometry.coordinates
0,Feature,41271,10471,수원문산고속도로,1000115,Suwon Munsan Expressway,20170714,1,1,0,...,78600.000,2000,고속국도 제17호 수원 문산선 노선명을 이용하여 명명,20170714,12,None,20170726,20170726094641,LineString,"[[126.89574250647738, 37.358390119674404], [12..."
1,Feature,41273,11551,엠티브이14로,3199103,Emtibeui 14-ro,20130425,3,3,1,...,179.000,20,시화MTV사업지구 명칭을 반영하여 안산시에서 부터 시작해서 일련번호방식 및 기초번호...,20130425,12,도로구간변경,20200406,20200406112843,LineString,"[[126.74290056773407, 37.30319380643736], [126..."
2,Feature,41271,12151,팔곡산단4길,4860543,Palgoksandan 4-gil,20210827,3,4,0,...,457.654,20,팔곡산업단지내 위치를 반영 용담로기준 우측지역지 일련번호를 사용,20210827,01,팔곡산업단지내 위치를 반영 용담로기준 우측지역지 일련번호를 사용,20210827,20210827095706,LineString,"[[126.88794507890543, 37.29018594623124], [126..."
3,Feature,41271,12091,팔곡산단1길,4860546,Palgoksandan 1-gil,20210827,3,4,0,...,383.555,20,팔곡일반산업단지 위치를 반영 용담로기준 좌측지역의 일련번호 사용,20210827,01,팔곡일반산업단지 위치를 반영 용담로기준 좌측지역의 일련번호 사용,20210827,20210827100021,LineString,"[[126.88669104695009, 37.28495384333724], [126..."
4,Feature,41271,12111,팔곡산단2길,4860545,Palgoksandan 2-gil,20210827,3,4,0,...,196.968,20,팔곡일반산업단지 위치를 적용 용담로기준 좌측지역의 일련번호 사용,20210827,01,팔곡일반산업단지 위치를 적용 용담로기준 좌측지역의 일련번호 사용,20210827,20210827095920,LineString,"[[126.88607787333206, 37.28680720191384], [126..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894,Feature,41273,11771,화정천자전거길,3353190,Hwajeongcheonjajeongeo-gil,20201207,3,3,0,...,5256.916,20,화정천을 따라 나 있는 자전거길의 특성을 반영함,20201207,01,화정천을 따라 나 있는 자전거길의 특성을 반영함,20201207,20201207090022,LineString,"[[126.81975831474612, 37.34936396932827], [126..."
1895,Feature,41273,11773,시우로,3191043,Siu-ro,20090112,3,3,1,...,380.985,20,예부터 시우동이라 불러왔던 지명유래를 반영,20090112,12,도로구간변경,20201120,20201120153014,LineString,"[[126.80432135603537, 37.30123658733514], [126..."
1896,Feature,41271,11951,충장로,3190071,Chungjang-ro,20090112,3,3,1,...,283.784,20,예전부터 불러왔던 도로명으로 주민들의 인지도가 매우 높은 특성을 반영함,20090112,12,도로구간변경,20210312,20210312105221,LineString,"[[126.85465491236525, 37.30939541035623], [126..."
1897,Feature,41271,12011,안산벌말길,4370326,Ansanbeolmal-gil,20090112,3,4,1,...,318.422,20,서해 바닷물이 들어오지 않아 넓은 벌판이 되자 벌말이라 불러왔다는 지명유래를 반영,20090112,12,도로구간변경,20210312,20210312105102,LineString,"[[126.8632282425455, 37.35734948626845], [126...."
